In [99]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [100]:
abund_tb=pd.read_csv("/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/data/PanBac/BFTMS23071402 (single)OTU (Table).csv")
trim_tbl=pd.read_csv("/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/data/Trim_summary.csv")

In [101]:
abund_tb.head(5)

,ID,Name,Taxonomy,Combined Abundance,Min,Max,Mean,Median,Std,BFTMS23071402 Abundance,...,BFTMS23071409 Abundance,BFTMS23071410 Abundance,BFTMS23071411 Abundance,BFTMS23071412 Abundance,BFTMS23071413 Abundance,BFTMS23071414 Abundance,BFTMS23071415 Abundance,BFTMS23071416 Abundance,BFTMS23071417 Abundance,Sequence
0,Staphylococcus epidermidis; mg|Staphylococcus|...,mg|Staphylococcus|epidermidis|-01,Bacteria; ; ; ; ; ; Staphylococcus epidermidis,19,0,7,1.1875,0.0,2.561738,0,...,6,7,0,0,0,0,0,0,0,TGATTTCTTAAAATACCTTTATGAAAAAGTACATGTTTTGGAAAAA...
1,Staphylococcus epidermidis; mg|Staphylococcus|...,mg|Staphylococcus|epidermidis|-02,Bacteria; ; ; ; ; ; Staphylococcus epidermidis,39,0,18,2.4375,0.0,5.303694,0,...,2,2,0,0,0,0,0,0,13,TTAATACTTGGTACATCAATTGTTTGAGACATATCTATAAAGTATT...
2,Pseudomonas aeruginosa; mg|Pseudomonas|aerugin...,mg|Pseudomonas|aeruginosa|MPAO1/P1|-01,Bacteria; ; ; ; ; ; Pseudomonas aeruginosa,132,0,122,8.2500,0.0,30.416005,0,...,0,9,122,1,0,0,0,0,0,TGCAGCGGACGCTGGCAGTATTCGTGGATACGCACGCCCTCGCGCA...
3,Pseudomonas aeruginosa; mg|Pseudomonas|aerugin...,mg|Pseudomonas|aeruginosa|MPAO1/P1|-02,Bacteria; ; ; ; ; ; Pseudomonas aeruginosa,22,0,22,1.3750,0.0,5.500000,0,...,0,0,22,0,0,0,0,0,0,CTTCCTCGGCTTCCACTTTTCCCGGCATTTCCCGGCCATCGCCGGT...
4,Citrobacter freundii; mg|Citrobacter|freundii|...,mg|Citrobacter|freundii|ATCC-8090--MTCC-1658|,Bacteria; ; ; ; ; ; Citrobacter freundii,2,0,1,0.1250,0.0,0.341565,0,...,0,0,0,0,1,0,0,1,0,GCGAAACGCACCGTGTATTGGGGTTGGAAATGGGGGCCGATGATTA...


In [102]:
# Rename the columns by removing the keyword abundance and BFTMS
abund_tb.columns = [col.replace("Abundance", '') for col in abund_tb.columns]
abund_tb.columns = [col.replace(" ", '') for col in abund_tb.columns]
#abund_tb.columns = [col.replace("BFTMS", '') for col in abund_tb.columns]

##add a control column that gets values from the spcified control sample
control="BFTMS23071417"
#abund_tb["control"]=abund_tb[control]

samples=trim_tbl["Name"].astype(str).tolist()
#samples.append('control')

abund_tb['reference_lib']='PanBac'

In [103]:
## mark species with high risk of false positive detection in a column "false_pos_prone"  

# first split the ID column into species group name and species ID
abund_tb[['species group', 'species id']] = abund_tb['ID'].str.split(';', expand=True)
## create a new column "false_pos_prone" and mark species with high risk of false positive detection
abund_tb['false_pos_prone']='No'


def mark_species(df, ID_column, false_pos_tag, species_list):
    # Update Column B to True for rows where values in Column A start with any of the strings in the search list
    for name in species_list:
        df.loc[df[ID_column].str.startswith(name), false_pos_tag] = 'Yes'
    return df

false_pos_prone_spicies=['Acinetobacter baumannii', 'Klebsiella oxytoca', 'Enterobacter cloacae',
                         'Citrobacter freundii']
abund_tb = mark_species(abund_tb, 'species group', 'false_pos_prone', false_pos_prone_spicies)
#abund_tb.loc[abund_tb['species group'].isin(false_pos_prone_spicies), 'false_pos_prone'] = 'Yes'

In [104]:
## data normalization and corection for control
control="BFTMS23071417"
resid_val=0.00 # this value will be added to sample and control to avoid deviding to 0

# normalize all abundances based on the total reads for each sample
for samp in samples:
    ## read the trimmed reads from trim_tbl and get the value based on the sample name
    trimmed_reads=trim_tbl[trim_tbl["Name"]==samp]["Trimmed sequences"].values[0]
    abund_tb[samp+"_Normalized"]=abund_tb[samp]*trimmed_reads/1000000

## correct normized abundance by deducing control abundance
for samp in samples:
    abund_tb[samp+"_Normalized_corrected"]=abund_tb[samp+"_Normalized"]-abund_tb[control+"_Normalized"]
    ## calculate signal to noize 
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

## ## calculate signal to noize
for samp in samples:
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

In [105]:
## check one sample and compare it against the results from manual analysis
sample='BFTMS23071405'
sample_df=abund_tb[['ID', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']]
sample_df.sort_values(by=sample+'_Normalized_corrected', ascending=False)
sample_df.head(5)

,ID,BFTMS23071405,BFTMS23071417,BFTMS23071405_Normalized,BFTMS23071417_Normalized,BFTMS23071405_Normalized_corrected,BFTMS23071405_S:N
0,Staphylococcus epidermidis; mg|Staphylococcus|...,0,0,0.000000,0.000000,0.000000,NaN
1,Staphylococcus epidermidis; mg|Staphylococcus|...,4,13,0.692804,1.309139,-0.616335,-0.470794
2,Pseudomonas aeruginosa; mg|Pseudomonas|aerugin...,0,0,0.000000,0.000000,0.000000,NaN
3,Pseudomonas aeruginosa; mg|Pseudomonas|aerugin...,0,0,0.000000,0.000000,0.000000,NaN
4,Citrobacter freundii; mg|Citrobacter|freundii|...,0,0,0.000000,0.000000,0.000000,NaN


In [106]:
## PanBac strict

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=500) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=10) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1))
                            ]
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='PanBac_strict'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/PanBac_strict.csv')




In [107]:
## PanBac moderate

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    
    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=300) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=10) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1))
                            ]
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='PanBac_moderate'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/PanBac_moderate.csv')



In [108]:
## PanBac loose

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    
    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=50) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=5) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=1))
                            ]

    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='PanBac_loose'
select_IDs.to_csv('/Users/hhakimjavadi/Dropbox (UFL)/microbiome_microbiology_CHLA/output_tables/PanBac_loose.csv')
